In [1]:
#!ls ~/trained_models/
from pathlib import Path
import pandas as pd
pd.set_option('display.max_colwidth', 20)   # JSON columns make things look weird in notebook without this

from datatype_recovery.models.dataset import load_dataset_from_path
from datatype_recovery.models.dataset.encoding import *


dataset_path = Path.home()/'datasets/tydamin_inmem'

# model_path = Path.home()/'trained_models/dragon_5hops_5heads_3linear__trainset_new_ep50.pt'

# eval_model_on_dataset(model_path, 'cuda:3', dataset_path)
dataset = load_dataset_from_path(dataset_path)

df = dataset.read_vars_csv()
df

,BinaryId,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,...,LeafSigned,LeafFloating,LeafSize,PtrLevels,PtrL1,PtrL2,PtrL3,Vartype,IsReturnType_Debug,IsReturnType_Strip
0,1000,1108027,"24,36,56,81,81,94",local_14,int32,stack,NaN,-20.0,BUILTIN,int32,...,True,False,4,LLL,L,L,L,l,False,False
1,1000,1108292,"24,36,56,81,81,95",local_14,int32,stack,NaN,-20.0,BUILTIN,int32,...,True,False,4,LLL,L,L,L,l,False,False
2,1000,1113799,"64,131",uVar1,uint32,register,eax,0.0,BUILTIN,uint32,...,True,False,4,LLL,L,L,L,l,False,False
3,1000,1114200,"125,316,326",local_c,uint32,stack,NaN,-12.0,BUILTIN,uint32,...,True,False,4,LLL,L,L,L,l,False,False
4,1000,1115123,"970,979",ppuVar3,uchar**,register,rax,0.0,PTR,"PTR,PTR,uchar",...,False,False,0,PLL,P,L,L,l,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556207,2496000,1130729,"35,211,222,228,2...",param_1,int32*,register,rdi,56.0,PTR,"PTR,int32",...,False,False,0,PLL,P,L,L,p,False,False
1556208,2496000,1130729,"42,169",param_2,int64,register,rsi,48.0,BUILTIN,int64,...,False,False,1,PLL,P,L,L,p,False,False
1556209,2496000,1130729,"118,255",param_3,uint64,register,rdx,16.0,BUILTIN,uint64,...,False,False,8,LLL,L,L,L,p,False,False
1556210,2496000,1131021,"23,116",param_1,uchar*,register,rdi,56.0,PTR,"PTR,uchar",...,False,False,1,PLL,P,L,L,p,False,False


In [2]:
#df.groupby('TypeSeq_Debug').count().BinaryId
#df.TypeSeq_Debug.unique()

In [3]:
train_split = 0.1   # training + validation (we subdivide this during training)
test_split = 0.1    # withhold this for independent test set after we're done training



Test set has 155,621 samples (10.0%)
Train set has 1,400,591 samples (90.0%)


In [63]:
from datatype_recovery.models.training import split_train_test

train_indices, test_indices = split_train_test(len(dataset), batch_size=3)

Dataset size is 1,556,212
Test set has 155,619 samples (10.0%)
Train set has 1,400,592 samples (90.0%)
1 samples unused due to batch alignment (batch_size = 3)


In [65]:
from torch.utils.data import Subset
from tqdm import tqdm

test_set = [x for x in tqdm(Subset(dataset, list(test_indices)), desc='Loading test set')]
train_set = [x for x in tqdm(Subset(dataset, list(train_indices)), desc='Loading training set')]

  0%|          | 0/155619 [00:00<?, ?it/s]

100%|██████████| 1400592/1400592 [00:54<00:00, 25569.04it/s]


In [67]:
#torch.save(test_set, Path.cwd()/'test.pt')

In [ ]:
torch.save(dataset.collate(test_set, Path.cwd()/'test2.pt'))